##Setup

In [47]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [48]:
import os
BASE_DIR = '/content/drive/MyDrive/Colab Notebooks/Deep-Learning-LW3'
DATA_DIR = os.path.join(BASE_DIR, "scraped_wiki_data")
DATA_FILE_DIR = os.path.join(DATA_DIR, "wiki-data-v2.csv")
USE_PICKELED_DATA = True

In [49]:
import cv2
import albumentations as A
import glob
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import matplotlib.patches as patches
import copy
import time
import random
import csv
from skimage import io, transform
import numpy as np
import pickle
import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

!pip install transformers
from transformers import ElectraModel, ElectraTokenizer, BertTokenizer, BertModel

!pip install nlu pyspark==3.0.2
import nlu
!pip install gensim
!pip install nltk

from sklearn.metrics.pairwise import cosine_similarity
from transformers import ElectraModel, ElectraConfig
from transformers import ElectraTokenizer, ElectraModel
import torch
from scipy import spatial
import io
from google.colab import files

In [50]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('Found GPU :', torch.cuda.get_device_name(0))

else:
    print('Not found, use CPU instead')
    device = torch.device("cpu")

Not found, use CPU instead


##Wikipedia Api usage

In [ ]:
!pip3 install wikipedia-api
!pip install beautifulsoup4
import wikipediaapi
import random
import time
import sys
import requests
from os.path import exists
import csv
import nltk.data
nltk.download('punkt')
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
class Wikipedia_scraper:
  scraped_lines_count = 0
  dropped_count = 0
  scraped_titles = []
  
  def __init__(self, timeout_in_secs_for_exception, pages_to_scrape_count, lines_to_scrape_per_page):
    self.wiki_api = wikipediaapi.Wikipedia('en')
    self.left_to_scrape_count = pages_to_scrape_count
    self.lines_to_scrape_per_page = lines_to_scrape_per_page
    self.timeout_in_secs_for_exception = timeout_in_secs_for_exception

  def scrape_random_pages(self):
    with open(DATA_FILE_DIR, 'a') as writefile:
      csvwriter = csv.writer(writefile)
      while(self.left_to_scrape_count > 0):
        # try:
          random_title = self.get_random_wiki_page_title()
          page = self.wiki_api.page(random_title)
          print(str(self.scraped_lines_count) + ". " + str(self.dropped_count) + ". " + page.title)
          if(not self.is_page_valid(page)):
            continue
          if page.title in self.scraped_titles:
            continue
          else:
            self.add_sentences_to_file(csvwriter, page.title, page.text)
            self.scraped_titles.append(page.title)
        # except:
        #   time.sleep(self.timeout_in_secs_for_exception)
        #   continue
      writefile.close()
        
  def is_page_valid(self, page):
    if( page == None ):
      return False
    if( not page.exists() ):
      return False
    if(page.links == None):
      return False
    return True

  def get_random_wiki_page_title(self):
    wiki_url_without_title = 'https://en.wikipedia.org/wiki/'
    random_page_url = requests.get('https://en.wikipedia.org/wiki/Special:Random').url
    return random_page_url.replace(wiki_url_without_title, "")
  
  def add_sentences_to_file(self, csvwriter, title, text):
    # sentences = text.split(".")
    sentences = nltk.tokenize.sent_tokenize(text)
    sentences_to_scrape_count = min(len(sentences), self.lines_to_scrape_per_page)
    for i in range(sentences_to_scrape_count):
      if(type(sentences[i]) != str or type(title) != str):
        self.dropped_count += 1
        return
      csvwriter.writerow([sentences[i], title])
      self.scraped_lines_count += 1
      self.left_to_scrape_count -= 1

scraper = Wikipedia_scraper(60, 10000, 5)
scraper.scrape_random_pages()

##NLP

###Dataset

In [51]:
col_names = ["sentence", "title"]

df = pd.read_csv(DATA_FILE_DIR, 
                delimiter=',' , 
                header= None,
                names= col_names)

In [52]:
def drop_all_invalid_sentences(df):
  done = False
  drop_count = 0

  while(not done):
    highest = 0
    done = True
    for i, sentence in enumerate(df.sentence):
      if(type(sentence) != str or len(sentence.split()) >= 512):
        df = df.drop(df.index[i])
        drop_count += 1
        done = False
        break
  print("dropped: " + str(drop_count))
  return df

df = drop_all_invalid_sentences(df)

dropped: 6


###----------

In [53]:
configuration = ElectraConfig()
model = ElectraModel(configuration)
configuration = model.config

In [54]:
tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
model = ElectraModel.from_pretrained("google/electra-small-discriminator")

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [55]:
def encode(string):
  inputs = tokenizer(string, return_tensors="pt")
  if(len(inputs.input_ids[0]) >= 512):
    return None
  return model(**inputs)[0].detach().squeeze()

class DataRow:
  def __init__(self, sentence, title, encoded):
    self.sentence = sentence
    self.title = title
    self.encoded = encoded

if(not USE_PICKELED_DATA):
  data = []
  outputs_done = 0
  for sentence, title in zip(df.sentence, df.title):
    encoded = encode(sentence)
    if(encoded != None):
      new_row = DataRow(
          sentence = sentence,
          title = title,
          encoded = encoded
      )
      data.append(new_row)
      outputs_done += 1
    
    if (outputs_done%1000 == 0):
      print(outputs_done)

  len(data)

In [56]:
if (not USE_PICKELED_DATA):
  pickle.dump(data, open('data.pkl', 'wb'))
  files.download('data.pkl') 
else:
  pickled_data_dir = os.path.join(BASE_DIR, "data.pkl")
  with open(pickled_data_dir, 'rb') as file_handle:
    data = pickle.load(file_handle)

In [57]:
len(data)

24931

##REST

###Imports

In [58]:
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken '286a98tsyq9sDorWLoOfjuttt0X_6ibv7NKYMTMPB7Rdfh8oH'
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import json

  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=9ab2e5f24b1e6d9eff594841689d0cef047688981e5b6e1589f6405f981336aa
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


###HTML

In [59]:
def render_input_form():
  return '''
    <!doctype html>
    <title>Search</title>
    <h1>Search text:</h1>
    <form method=post>
      <input type="text" name="input-text" placeholder="search"></input>
    </form>'''

def render_results(result, url, similarity):
  return '''
    <!doctype html>
    <title>Results</title>
    <h1>Results</h1>
    <div>{sentence}</div>
    <h1>Similarity</h1>
    <div>{similarity}</div>
    <a href={url}><h1>Go to the wiki page</h1></a>'''.format(sentence = result, url = url, similarity = similarity)

###----------

In [60]:
def make_prediction(sentence, find_similar=True):
  encoded_sentence = encode(sentence)
  cos_similarity = nn.CosineSimilarity(dim=0, eps=1e-6)

  max_sim = 0
  min_sim = 1
  for row in data:
    similarity = cos_similarity(row.encoded[0], encoded_sentence[0])
    if(similarity > max_sim and find_similar):
      max_sim = similarity
      pred_row = row
    if(similarity < min_sim and not find_similar):
      min_sim = similarity
      pred_row = row

  if(find_similar):
    sim = max_sim
  else:
    sim = min_sim

  result = type('result', (object,), {
    'similarity': sim,
    "title": pred_row.title,
    "sentence": pred_row.sentence
  })()

  return result

In [61]:
def get_url_from_title(title):
  title = title.replace(" ", "_")
  return "https://en.wikipedia.org/wiki/" + title

In [62]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/similarity', methods=['GET', 'POST'])
def similarity():
  if request.method == 'POST':
    query_string = request.form['input-text']

    pred = make_prediction(query_string, True)
    url = get_url_from_title(pred.title)

    return render_results(pred.sentence, url, pred.similarity)  
  return render_input_form()

@app.route('/unsimilarity', methods=['GET', 'POST'])
def unsimilarity():
  if request.method == 'POST':
    query_string = request.form['input-text']

    pred = make_prediction(query_string, False)
    url = get_url_from_title(pred.title)

    return render_results(pred.sentence, url, pred.similarity)  
  return render_input_form()

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://357c-34-73-35-233.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/May/2022 09:15:31] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2022 09:15:33] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2022 09:15:53] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2022 09:15:53] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2022 09:16:01] "GET /similarity HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 09:16:01] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2022 09:16:02] "GET /similarity HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 09:16:22] "POST /similarity HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 09:17:00] "GET /similarity HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 09:17:08] "POST /similarity HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 09:17:09] "GET /similarity HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 09:17:20] "POST /similarity HTTP/1.1" 200 -
